author: G.Seijo (giovanni.seijo@colorado.edu)

Modified code from: James Simkins (https://github.com/jsimkins2/nwa25) \
This code reads mom surface output, plots gridded maps of the variable of choice and generates a gif.\
All plotting parameters are defined at top of the plot below the. "Inputs" title.

In [2]:
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from datetime import datetime
import os.path
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
from PIL import Image
import glob
import sys
import cmocean.cm as cmo
import time
import matplotlib.image as image
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

In [7]:
#Directories:

#directory were the input file is found:
in_dir = '/glade/work/gseijo/CARIB_025.007.glofas_proc/'
#directory to output images and gif:
out_dir = '/glade/work/gseijo/CARIB_025.007.glofas_proc/sss/'
#Open the input file:
ds = xr.open_dataset(in_dir + 'CARIB_025.007.glofas_sfc.nc')
#name of google-earth like background for plotting (bluemarble.png), the code assumes the file lives in the same directory as the notebook:
earth = 'bluemarble.png'
#case name (used on output filenames and figure titles:
case_nm = 'CARIB_025.007' 
#choose variable to plot:
var2plot = 'sos' # examples: 'SSH','sos',tos
plotSSH_contour = True #will plot SSH contours (not filled) on top of var2plot
fig_width = 12
fig_height = 12
title_fntsz = 16
cb_fntsz = 12

In [8]:
if var2plot == 'sos':
    bfr = ds.sos
    cmap_var = 'cmo.haline'
    varmin = 30
    varmax = 38
    cb_ticks = [30,32,34,36,38]  
    cb_shrink = 0.5
    cb_label = 'SSS, psu'
    fig_title = str('MOM6 '+ case_nm + ' SSS [psu]')
    fig_outname = str(case_nm + '_sss_')
    gif_outname = str(case_nm + '_sss_2000-2009')

elif var2plot == 'tos':
    bfr = ds.tos
    cmap_var = 'cmo.thermal'
    varmin = 14
    varmax = 28
    cb_ticks = [14,16,18,20,22,24,26,28] 
    cb_shrink = 0.5
    cb_label = 'SST, $^\circ$C'
    fig_title = str('MOM6 '+ case_nm + ' SST [$^\circ$C]')
    fig_outname = str(case_nm + '_sst_')
    gif_outname = str(case_nm + '_sst_2000-2009')

    
elif var2plot == 'SSH':
    bfr = ds.SSH
    cmap_var = 'cmo.balance'
    varmin = -0.5
    varmax = 0.5
    cb_ticks = [-.5,-.25,0,.25,.5] 
    cb_shrink = 0.5
    cb_label = 'SSH, m'
    fig_title = str('MOM6 '+ case_nm + ' SSH [m]')
    fig_outname = str(case_nm + '_ssh_')
    gif_outname = str(case_nm + '_ssh_2000-2009')

    
if plotSSH_contour:
    bfr_contours = ds.SSH
    num_contourlev = 12
    contour_colors = 'black'
    contour_fntsz = 8
    
print('var2plot: ' ,var2plot)


var2plot:  sos


In [9]:
img = plt.imread('./'+earth)
img_extent = (-180, 180, -90, 90)

for ii in range (np.size(ds.time[:])):
    figure = plt.figure(figsize=(fig_width,fig_height))
    left = 0.01
    bottom = 0.01
    width = 0.99
    height = 0.99
    ax1 = plt.axes([left, bottom, width, height], projection=ccrs.PlateCarree(), extent=[-100, -33, -5, 30.1], facecolor='black')
    ax1.imshow(img, origin='upper', extent=img_extent, transform=ccrs.PlateCarree(),vmin=0,vmax=200)
    im = ax1.pcolormesh(ds.xh.values, ds.yh.values, bfr[ii].values, cmap=cmap_var, vmin=varmin, vmax=varmax)
    cb = plt.colorbar(im, ticks=cb_ticks,shrink=cb_shrink)
    cb.ax.tick_params(labelsize= cb_fntsz)
    cb.ax.yaxis.set_tick_params(color='k')
    cb.set_label(cb_label, color='k', fontsize=cb_fntsz,fontweight="bold")
    for tick in cb.ax.yaxis.get_major_ticks():
        tick.label2.set_fontweight('bold')
    if plotSSH_contour:
        contours = plt.contour(ds.xh.values,ds.yh.values,bfr_contours[ii].values,num_contourlev,colors=contour_colors)
        plt.clabel(contours, inline=True, fontsize=contour_fntsz)
    plt.title(str(fig_title + ' -' + str(ds.time[ii].values)), fontsize=title_fntsz, color='k',fontweight="bold")
    gl = ax1.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True, x_inline=False, y_inline=False)
    gl.top_labels = False
    gl.left_labels = False
    gl.right_labels=True
    gl.bottom_labels=True
    gl.xlines = True
    gl.ylines = True
    gl.xlocator = mticker.FixedLocator([-100,-90,-80,-70, -60, -50, -40])
    gl.ylocator = mticker.FixedLocator([-10, 0, 10, 20, 30, 40, 50])
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size':12,'color': 'black', 'weight': 'bold'}
    gl.ylabel_style = {'size':12,'color': 'black', 'weight': 'bold'}
    cb.ax.yaxis.set_tick_params(color='k')
    plt.setp(plt.getp(cb.ax.axes, 'yticklabels'), color='k')
    
    plt.savefig(out_dir + fig_outname + str(ds.time[ii].values) + '.png')
    plt.close()
    
#generate a gif animation
img_names = sorted(glob.glob(out_dir+"*.png"))
dur_vals = []
for i in range(0,len(img_names)):
    if i != len(img_names)-1:
        dur_vals.append(200)
dur_vals.append(3000)

img, *imgs = [Image.open(f) for f in img_names]
img.save(fp= out_dir + gif_outname + '.gif', format='GIF', append_images=imgs,
     save_all=True, duration=dur_vals, loop=0)

In [1]:
#can use convert on terminal: convert -delay 50 *.png out.gif